# Scrape Pandas Table

In [6]:
from splinter import Browser
from bs4 import BeautifulSoup
import requests
import webbot as web
import pandas as pd
#import io

In [7]:
# Read in HTML tables into a DataFrame
browser = Browser('chrome')

In [10]:
# Select the second table
url = 'https://www.updinc.net/accounts/signin/'
browser.visit(url)

In [21]:
#user="carpediemimports.contact@gmail.com"
#password="Kamilla2024"

In [11]:
url2='https://www.updinc.net/catalog/?catid=3&pageNum=1'
browser.visit(url2)

In [12]:
def get_summary():
    apparel_list = []
    page_num = 1
    visited_UPCs = set()  # Set to keep track of visited UPCs

    while True:
        # Visit the URL for the current page
        browser.visit(f"https://www.updinc.net/catalog/?catid=3&pageNum={page_num}")
        
        # Get the HTML content
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        
        # Find the product boxes
        new_apparels = soup.find_all('div', class_='prdct-box')

        if not new_apparels:
            break

        for new_apparel in new_apparels:
            name = new_apparel.find("div", class_="prdct-box2").text.strip()
            UPC = new_apparel.find('div', class_='prdct-box3').text.strip()

            # Check if UPC has been visited before
            if UPC not in visited_UPCs:
                summary_dict = {'name': name, 'UPC': UPC}
                apparel_list.append(summary_dict)
                visited_UPCs.add(UPC)  # Add UPC to visited set
            
        # Move to the next page
        page_num += 1
    
    return apparel_list

In [13]:
apparels_list=[]
for _ in range(1):
    summary=get_summary()
    apparels_list.extend(get_summary())
    
apparels_list

[{'name': 'Wholesale Ladies Assorted Designs Slipper Socks  Size 9-11 Pre priced $6.99\nItem: 14BIGFSS\n$1.49 EachSelect Quantity:\n\n\n\xa010\xa0\n\xa040\xa0\n\xa070\xa0\n\n\n\xa020\xa0\n\xa050\xa0\n\xa080\xa0\n\n\n\xa030\n\xa060\n\xa090\n\n\nor Enter Quantity:',
  'UPC': 'PRODUCT DETAILS\nItem Size: 5" x 3" x 14"\nCase: 10\nInner: \nInner Size: \nUPC: 081715979249...\nWT: 4\nCF: 0.85\nSub-Category: AP03\nOrigin: China'},
 {'name': 'Wholesale Frozen "Anna" Slipper Sock Size 3T-4T\nItem: 193159031794\n$0.98 EachSelect Quantity:\n\n\n\xa012\xa0\n\xa048\xa0\n\xa084\xa0\n\n\n\xa024\xa0\n\xa060\xa0\n\xa096\xa0\n\n\n\xa036\n\xa072\n\xa0108\n\n\nor Enter Quantity:',
  'UPC': 'PRODUCT DETAILS\nItem Size: 3.5" x .50" x 7"\nCase: 12\nInner: \nInner Size: \nUPC: 193159031794\nWT: 1\nCF: 0.16\nSub-Category: AP02\nOrigin: China'},
 {'name': 'Wholesale Frozen "Anna" Slipper Sock Size 2T-3T\nItem: 193159031848\n$0.98 EachSelect Quantity:\n\n\n\xa012\xa0\n\xa048\xa0\n\xa084\xa0\n\n\n\xa024\xa0\n\xa06

In [14]:
print(f"Scraped {len(apparels_list)} records.")

#Convert the list of dictionaries to a DataFrame

df= pd.DataFrame(apparels_list)
df.head()

Scraped 159 records.


,name,UPC
0,Wholesale Ladies Assorted Designs Slipper Sock...,"PRODUCT DETAILS\nItem Size: 5"" x 3"" x 14""\nCas..."
1,"Wholesale Frozen ""Anna"" Slipper Sock Size 3T-4...","PRODUCT DETAILS\nItem Size: 3.5"" x .50"" x 7""\n..."
2,"Wholesale Frozen ""Anna"" Slipper Sock Size 2T-3...","PRODUCT DETAILS\nItem Size: 3.25"" x .75"" x 6.5..."
3,"Wholesale Pittsburgh Steelers 50""x60"" Fleece T...","PRODUCT DETAILS\nItem Size: 6"" x 6"" x 10""\nCas..."
4,Wholesale LOL Surprise Soft Plush Slippers Siz...,"PRODUCT DETAILS\nItem Size: 3"" x 3.25"" x 9""\nC..."


In [15]:
# Concatenate 'name' and 'UPC' columns with a separator
df['concatenated'] = df['name'] + '\n' + df['UPC']

# Calculate the maximum number of elements in any row after splitting
max_len = max(len(row.split('\n')) for row in df['concatenated'])

# Split the data into separate columns and pad shorter rows with None values
data_list = [row.split('\n') + [None] * (max_len - len(row.split('\n'))) for row in df['concatenated'] if row.strip()]

# Create a DataFrame from the split data
new_df = pd.DataFrame(data_list)

# Rename the columns
new_df.columns = [f'Column_{i+1}' for i in range(max_len)]

# Print the new DataFrame
new_df.head()



,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,...,Column_24,Column_25,Column_26,Column_27,Column_28,Column_29,Column_30,Column_31,Column_32,Column_33
0,Wholesale Ladies Assorted Designs Slipper Sock...,Item: 14BIGFSS,$1.49 EachSelect Quantity:,,,10,40,70,,,...,Case: 10,Inner:,Inner Size:,UPC: 081715979249...,WT: 4,CF: 0.85,Sub-Category: AP03,Origin: China,None,None
1,"Wholesale Frozen ""Anna"" Slipper Sock Size 3T-4T",Item: 193159031794,$0.98 EachSelect Quantity:,,,12,48,84,,,...,Case: 12,Inner:,Inner Size:,UPC: 193159031794,WT: 1,CF: 0.16,Sub-Category: AP02,Origin: China,None,None
2,"Wholesale Frozen ""Anna"" Slipper Sock Size 2T-3T",Item: 193159031848,$0.98 EachSelect Quantity:,,,12,48,84,,,...,Case: 12,Inner:,Inner Size:,UPC: 193159031848,WT: 1,CF: 0.14,Sub-Category: AP02,Origin: China,None,None
3,"Wholesale Pittsburgh Steelers 50""x60"" Fleece T...",Item: 1NFL031020078,$5.45 EachSelect Quantity:,,,8,32,56,,,...,Case: 8,Inner:,Inner Size:,UPC: 037918682351,WT: 8,CF: 1.33,Sub-Category: AP05,Origin: China,None,None
4,Wholesale LOL Surprise Soft Plush Slippers Siz...,Item: 37583LLUP,$3.58 EachSelect Quantity:,,,12,48,84,,,...,Case: 12,Inner:,Inner Size:,UPC: 752229149090...,WT: 3.91,CF: 1.22,Sub-Category: AP02,Origin: USA,None,None


In [16]:
#Getting necessary columns
new_df2= new_df[['Column_1', 'Column_2', 'Column_3', 'Column_24', 'Column_27']]

#Rename the columns
new_column_names = {
    'Column_1': 'Item_Name',
    'Column_2': 'Item_Number',
    'Column_3': 'Price',
    'Column_24': 'Quantity',
    'Column_27': 'UPC'
}

new_df2.rename(columns=new_column_names, inplace=True)

# Remove 'Wholesale' from the column 'Item_Name'
new_df2['Item_Name'] = new_df2['Item_Name'].str.replace('Wholesale', '')

#Remove 'Item:' from the column 'Item_Number'
new_df2['Item_Number'] = new_df2['Item_Number'].str.replace('Item:', '')

#Remove 'EachSelect Quantity:' from the column 'Price'
new_df2['Price'] = new_df2['Price'].str.replace('EachSelect Quantity:', '')

#Remove 'UPC' from the column 'UPC'
new_df2['UPC'] = new_df2['UPC'].str.replace('UPC:', '')

#Remove 'EachSelect Quantity:' from the column 'Price'
new_df2['Quantity'] = new_df2['Quantity'].str.replace('Case:', '')

new_df2.head()

C:\Users\Fernanda\AppData\Local\Temp\ipykernel_11576\661018309.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df2.rename(columns=new_column_names, inplace=True)
C:\Users\Fernanda\AppData\Local\Temp\ipykernel_11576\661018309.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_df2['Item_Name'] = new_df2['Item_Name'].str.replace('Wholesale', '')
C:\Users\Fernanda\AppData\Local\Temp\ipykernel_11576\661018309.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value

,Item_Name,Item_Number,Price,Quantity,UPC
0,Ladies Assorted Designs Slipper Socks Size 9...,14BIGFSS,$1.49,10,081715979249...
1,"Frozen ""Anna"" Slipper Sock Size 3T-4T",193159031794,$0.98,12,193159031794
2,"Frozen ""Anna"" Slipper Sock Size 2T-3T",193159031848,$0.98,12,193159031848
3,"Pittsburgh Steelers 50""x60"" Fleece Throw",1NFL031020078,$5.45,8,037918682351
4,LOL Surprise Soft Plush Slippers Size 11-3,37583LLUP,$3.58,12,752229149090...


In [17]:

# Check for duplicates based on the specified columns
duplicates = new_df2[new_df2.duplicated(subset=['Item_Name','Item_Number', 'Price', 'UPC'], keep=False)]

# Print the duplicates
print("Duplicate rows based on the specified columns:")
duplicates.head()

Duplicate rows based on the specified columns:


,Item_Name,Item_Number,Price,Quantity,UPC


In [18]:
# Drop duplicates based on the UPC column
new_df2.drop_duplicates(duplicates)

# Print the DataFrame after dropping duplicates
new_df2.head()

,Item_Name,Item_Number,Price,Quantity,UPC
0,Ladies Assorted Designs Slipper Socks Size 9...,14BIGFSS,$1.49,10,081715979249...
1,"Frozen ""Anna"" Slipper Sock Size 3T-4T",193159031794,$0.98,12,193159031794
2,"Frozen ""Anna"" Slipper Sock Size 2T-3T",193159031848,$0.98,12,193159031848
3,"Pittsburgh Steelers 50""x60"" Fleece Throw",1NFL031020078,$5.45,8,037918682351
4,LOL Surprise Soft Plush Slippers Size 11-3,37583LLUP,$3.58,12,752229149090...


In [19]:
#Changing the column oreder
# Define the new sequence of column names
new_column_order = ['Item_Number','Item_Name', 'Price', 'Quantity', 'UPC' ]

# Reorder the columns based on the new sequence
final_dataFrame = new_df2[new_column_order]
final_dataFrame.head()

,Item_Number,Item_Name,Price,Quantity,UPC
0,14BIGFSS,Ladies Assorted Designs Slipper Socks Size 9...,$1.49,10,081715979249...
1,193159031794,"Frozen ""Anna"" Slipper Sock Size 3T-4T",$0.98,12,193159031794
2,193159031848,"Frozen ""Anna"" Slipper Sock Size 2T-3T",$0.98,12,193159031848
3,1NFL031020078,"Pittsburgh Steelers 50""x60"" Fleece Throw",$5.45,8,037918682351
4,37583LLUP,LOL Surprise Soft Plush Slippers Size 11-3,$3.58,12,752229149090...


In [20]:
final_dataFrame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 159 entries, 0 to 158
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Item_Number  159 non-null    object
 1   Item_Name    159 non-null    object
 2   Price        159 non-null    object
 3   Quantity     159 non-null    object
 4   UPC          159 non-null    object
dtypes: object(5)
memory usage: 6.3+ KB


In [22]:
# Write DataFrame to a CSV file
final_dataFrame.to_excel('Resources/apparel.xlsx', index=False)

In [24]:
def get_summary():
    baby_list = []
    page_num = 1
    visited_UPCs = set()  # Set to keep track of visited UPCs

    while True:
        # Visit the URL for the current page
        browser.visit(f"https://www.updinc.net/catalog/?catid=10&pageNum={page_num}")
        
        # Get the HTML content
        html = browser.html
        soup = BeautifulSoup(html, 'html.parser')
        
        # Find the product boxes
        found_babies = soup.find_all('div', class_='prdct-box')

        if not found_babies:
            break

        for baby in found_babies:
            name = baby.find("div", class_="prdct-box2").text.strip()
            UPC = baby.find('div', class_='prdct-box3').text.strip()

            # Check if UPC has been visited before
            if UPC not in visited_UPCs:
                summary_dict = {'name': name, 'UPC': UPC}
                baby_list.append(summary_dict)
                visited_UPCs.add(UPC)  # Add UPC to visited set
            
        # Move to the next page
        page_num += 1
    
    return baby_list

In [25]:
babies_list=[]
for _ in range(1):
    summary=get_summary()
    babies_list.extend(get_summary())
    
babies_list

[{'name': 'Wholesale Little Tikes Fish & Chips Action Game\nItem: 13422\n$5.45 EachSelect Quantity:\n\n\n\xa04\xa0\n\xa016\xa0\n\xa028\xa0\n\n\n\xa08\xa0\n\xa020\xa0\n\xa032\xa0\n\n\n\xa012\n\xa024\n\xa036\n\n\nor Enter Quantity:',
  'UPC': 'PRODUCT DETAILS\nItem Size: 8" x 5" x 10.5"\nCase: 4\nInner: \nInner Size: \nUPC: 686141134226\nWT: 4\nCF: 1.19\nSub-Category: BB02\nOrigin: China'},
 {'name': 'Wholesale Little Tikes Milk & Cookies Action Game\nItem: 13424\n$5.45 EachSelect Quantity:\n\n\n\xa04\xa0\n\xa016\xa0\n\xa028\xa0\n\n\n\xa08\xa0\n\xa020\xa0\n\xa032\xa0\n\n\n\xa012\n\xa024\n\xa036\n\n\nor Enter Quantity:',
  'UPC': 'PRODUCT DETAILS\nItem Size: 8" x 5" x 10.5"\nCase: 4\nInner: \nInner Size: \nUPC: 686141134240\nWT: 5\nCF: 1.29\nSub-Category: BB02\nOrigin: China'},
 {'name': 'Wholesale Itsy Tots Baby Activity Walker with sound in color box\nItem: 182209\n$14.25 EachSelect Quantity:\n\n\n\xa06\xa0\n\xa024\xa0\n\xa042\xa0\n\n\n\xa012\xa0\n\xa030\xa0\n\xa048\xa0\n\n\n\xa018\n\xa

In [26]:
print(f"Scraped {len(babies_list)} records.")
baby_list_df= pd.DataFrame(babies_list)
baby_list_df.head()

Scraped 54 records.


,name,UPC
0,Wholesale Little Tikes Fish & Chips Action Gam...,"PRODUCT DETAILS\nItem Size: 8"" x 5"" x 10.5""\nC..."
1,Wholesale Little Tikes Milk & Cookies Action G...,"PRODUCT DETAILS\nItem Size: 8"" x 5"" x 10.5""\nC..."
2,Wholesale Itsy Tots Baby Activity Walker with ...,"PRODUCT DETAILS\nItem Size: 18"" x 4.25"" x 13.2..."
3,Wholesale Itsy Tots Junior Mechanic Plane in w...,"PRODUCT DETAILS\nItem Size: 7"" x 5.5"" x 5.5""\n..."
4,Wholesale Safety 1st 2 Pk Glow In The Dark Pac...,"PRODUCT DETAILS\nItem Size: 4"" x 2"" x 4.5""\nCa..."


In [28]:
# Concatenate 'name' and 'UPC' columns with a separator
baby_list_df['concatenated'] = baby_list_df['name'] + '\n' + baby_list_df['UPC']

# Calculate the maximum number of elements in any row after splitting
max_len = max(len(row.split('\n')) for row in baby_list_df['concatenated'])

# Split the data into separate columns and pad shorter rows with None values
data_list2 = [row.split('\n') + [None] * (max_len - len(row.split('\n'))) for row in baby_list_df['concatenated'] if row.strip()]

# Create a DataFrame from the split data
baby_list_data = pd.DataFrame(data_list2)

# Rename the columns
baby_list_data.columns = [f'Column_{i+1}' for i in range(max_len)]

# Print the new DataFrame
baby_list_data.head()

,Column_1,Column_2,Column_3,Column_4,Column_5,Column_6,Column_7,Column_8,Column_9,Column_10,...,Column_22,Column_23,Column_24,Column_25,Column_26,Column_27,Column_28,Column_29,Column_30,Column_31
0,Wholesale Little Tikes Fish & Chips Action Game,Item: 13422,$5.45 EachSelect Quantity:,,,4,16,28,,,...,PRODUCT DETAILS,"Item Size: 8"" x 5"" x 10.5""",Case: 4,Inner:,Inner Size:,UPC: 686141134226,WT: 4,CF: 1.19,Sub-Category: BB02,Origin: China
1,Wholesale Little Tikes Milk & Cookies Action Game,Item: 13424,$5.45 EachSelect Quantity:,,,4,16,28,,,...,PRODUCT DETAILS,"Item Size: 8"" x 5"" x 10.5""",Case: 4,Inner:,Inner Size:,UPC: 686141134240,WT: 5,CF: 1.29,Sub-Category: BB02,Origin: China
2,Wholesale Itsy Tots Baby Activity Walker with ...,Item: 182209,$14.25 EachSelect Quantity:,,,6,24,42,,,...,PRODUCT DETAILS,"Item Size: 18"" x 4.25"" x 13.2""",Case: 6,Inner:,Inner Size:,UPC: 810076720470,WT: 21,CF: 4.16,Sub-Category: BB02,Origin: China
3,Wholesale Itsy Tots Junior Mechanic Plane in w...,Item: 2010,$3.91 EachSelect Quantity:,,,12,48,84,,,...,PRODUCT DETAILS,"Item Size: 7"" x 5.5"" x 5.5""",Case: 12,Inner:,Inner Size:,UPC: 678634883056,WT: 9,CF: 1.77,Sub-Category: BB02,Origin: China
4,Wholesale Safety 1st 2 Pk Glow In The Dark Pac...,Item: 20973,$1.27 EachSelect Quantity:,,,36,144,252,,,...,PRODUCT DETAILS,"Item Size: 4"" x 2"" x 4.5""",Case: 36,Inner:,Inner Size:,UPC: 810002209734,WT: 5,CF: 0.68,Sub-Category: BB01,Origin: China


In [29]:
#Getting necessary columns
new_baby_list_df= baby_list_data[['Column_1', 'Column_2', 'Column_3', 'Column_24', 'Column_27']]

#Rename the columns
new_column_names = {
    'Column_1': 'Item_Name',
    'Column_2': 'Item_Number',
    'Column_3': 'Price',
    'Column_24': 'Quantity',
    'Column_27': 'UPC'
}

new_baby_list_df.rename(columns=new_column_names, inplace=True)

# Remove 'Wholesale' from the column 'Item_Name'
new_baby_list_df['Item_Name'] = new_baby_list_df['Item_Name'].str.replace('Wholesale', '')

#Remove 'Item:' from the column 'Item_Number'
new_baby_list_df['Item_Number'] = new_baby_list_df['Item_Number'].str.replace('Item:', '')

#Remove 'EachSelect Quantity:' from the column 'Price'
new_baby_list_df['Price'] = new_baby_list_df['Price'].str.replace('EachSelect Quantity:', '')

#Remove 'UPC' from the column 'UPC'
new_baby_list_df['UPC'] = new_baby_list_df['UPC'].str.replace('UPC:', '')

#Remove 'UPC' from the column 'UPC'
new_baby_list_df['Quantity'] = new_baby_list_df['Quantity'].str.replace('Case:', '')

new_baby_list_df.head()

C:\Users\Fernanda\AppData\Local\Temp\ipykernel_11576\1844083878.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_baby_list_df.rename(columns=new_column_names, inplace=True)
C:\Users\Fernanda\AppData\Local\Temp\ipykernel_11576\1844083878.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  new_baby_list_df['Item_Name'] = new_baby_list_df['Item_Name'].str.replace('Wholesale', '')
C:\Users\Fernanda\AppData\Local\Temp\ipykernel_11576\1844083878.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[ro

,Item_Name,Item_Number,Price,Quantity,UPC
0,Little Tikes Fish & Chips Action Game,13422,$5.45,4,686141134226
1,Little Tikes Milk & Cookies Action Game,13424,$5.45,4,686141134240
2,Itsy Tots Baby Activity Walker with sound in ...,182209,$14.25,6,810076720470
3,Itsy Tots Junior Mechanic Plane in window box,2010,$3.91,12,678634883056
4,Safety 1st 2 Pk Glow In The Dark Pacifier W/ ...,20973,$1.27,36,810002209734


In [30]:
# Check for duplicates based on the specified columns
duplicates2 = new_baby_list_df[new_baby_list_df.duplicated(subset=['Item_Name','Item_Number', 'Price', 'UPC'], keep=False)]

# Print the duplicates
print("Duplicate rows based on the specified columns:")
duplicates2.head()

Duplicate rows based on the specified columns:


,Item_Name,Item_Number,Price,Quantity,UPC


In [31]:
# Drop duplicates based on the UPC column
new_baby_list_df.drop_duplicates(duplicates2)

# Print the DataFrame after dropping duplicates
new_baby_list_df.head()

,Item_Name,Item_Number,Price,Quantity,UPC
0,Little Tikes Fish & Chips Action Game,13422,$5.45,4,686141134226
1,Little Tikes Milk & Cookies Action Game,13424,$5.45,4,686141134240
2,Itsy Tots Baby Activity Walker with sound in ...,182209,$14.25,6,810076720470
3,Itsy Tots Junior Mechanic Plane in window box,2010,$3.91,12,678634883056
4,Safety 1st 2 Pk Glow In The Dark Pacifier W/ ...,20973,$1.27,36,810002209734


In [32]:
new_baby_list_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54 entries, 0 to 53
Data columns (total 5 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Item_Name    54 non-null     object
 1   Item_Number  54 non-null     object
 2   Price        54 non-null     object
 3   Quantity     54 non-null     object
 4   UPC          54 non-null     object
dtypes: object(5)
memory usage: 2.2+ KB


In [33]:
#Changing the column oreder
# Define the new sequence of column names
new_column_order = ['Item_Number','Item_Name', 'Price', 'Quantity', 'UPC' ]

# Reorder the columns based on the new sequence
final_dataFrame2 = new_baby_list_df[new_column_order]
final_dataFrame2.head()

,Item_Number,Item_Name,Price,Quantity,UPC
0,13422,Little Tikes Fish & Chips Action Game,$5.45,4,686141134226
1,13424,Little Tikes Milk & Cookies Action Game,$5.45,4,686141134240
2,182209,Itsy Tots Baby Activity Walker with sound in ...,$14.25,6,810076720470
3,2010,Itsy Tots Junior Mechanic Plane in window box,$3.91,12,678634883056
4,20973,Safety 1st 2 Pk Glow In The Dark Pacifier W/ ...,$1.27,36,810002209734


In [34]:
# Write DataFrame to a CSV file
final_dataFrame2.to_excel('Resources/baby.xlsx', index=False)